# Setup

Install the intersection example.

**Note**: The runtime needs to be restarted after installing the dependencies, hence `os.kill(os.getpid(), 9)` is added to stop the current session. Please ignore any resulting error message and simply continue to execute the subsequent cells as per normal.

In [ ]:
%%capture --no-stderr
!git clone https://github.com/huawei-noah/SMARTS /content/SMARTS
!cd /content/SMARTS && git checkout intersection-v0 && cd /content/SMARTS/examples/rl/intersection && pip install --force-reinstall .
import sys
sys.path.insert(0, "/content/SMARTS/")
import os
os.kill(os.getpid(), 9)

In [ ]:
# Develop reinforcement learning code

We begin by building the necessary environment wrappers. Firstly, an info wrapper is built to help log instances when the ego agent succesfully completes an unprotected left turn.

In [ ]:
from typing import Any, Dict, Tuple

import gym


class Info(gym.Wrapper):
    def __init__(self, env: gym.Env):
        super(Info, self).__init__(env)

    def step(self, action: Any) -> Tuple[Any, float, bool, Dict[str, Any]]:
        """Steps the environment. A new "is_success" key is added to the
        returned `info`.

        Args:
            action (Any): Action for the agent.

        Returns:
            Tuple[ Any, float, bool, Dict[str, Any] ]:
                Observation, reward, done, and info, for the agent is returned.
        """
        obs, reward, done, info = self.env.step(action)
        info["is_success"] = bool(info["score"])

        return obs, reward, done, info

The `intersection-v0` environment has a continuous action space of `gym.spaces.Box(low=-1.0, high=1.0, shape=(3,), dtype=np.float32)` described by
+ Throttle: [0,1]
+ Brake: [0,1]
+ Steering: [-1,1]

In order to build a simple reinforcement learning policy, we discretise the action space using an action wrapper.

In [ ]:
from typing import Callable, Tuple

import gym
import numpy as np


class Action(gym.ActionWrapper):
    def __init__(self, env: gym.Env):
        super().__init__(env)
        self._wrapper, self.action_space = _discrete()

    def action(self, action):
        """Adapts the action input to the wrapped environment.

        Note: Users should not directly call this method.
        """
        wrapped_act = self._wrapper(action)
        return wrapped_act


def _discrete() -> Tuple[Callable[[int], np.ndarray], gym.Space]:
    space = gym.spaces.Discrete(n=4)

    action_map = {
        # key: [throttle, brake, steering]
        0: [0.3, 0, 0],  # keep_direction
        1: [0, 1, 0],  # slow_down
        2: [0.3, 0, -0.5],  # turn_left
        3: [0.3, 0, 0.5],  # turn_right
    }

    def wrapper(model_action: int) -> np.ndarray:
        throttle, brake, steering = action_map[model_action]
        return np.array([throttle, brake, steering], dtype=np.float32)

    return wrapper, space

In [ ]:
Next, we define the rewards using a reward wrapper. The agent is rewarded based on the distance travelled (in meters) per step and is penalised when it collides, goes off-road, goes off-route, goes wrong-way, or drives on the road shoulder.

from typing import Dict

import gym
import numpy as np


class Reward(gym.Wrapper):
    def __init__(self, env: gym.Env):
        super().__init__(env)

    def reset(self, **kwargs):
        return self.env.reset(**kwargs)

    def step(self, action):
        """Adapts the wrapped environment's step.

        Note: Users should not directly call this method.
        """
        obs, reward, done, info = self.env.step(action)
        wrapped_reward = self._reward(obs, reward)

        if done:
            if obs["events"]["reached_goal"]:
                print(f"ENV: Hooray! Vehicle reached goal.")
            elif obs["events"]["reached_max_episode_steps"]:
                print(f"ENV: Vehicle reached max episode steps.")
            elif (
                obs["events"]["off_road"]
                | obs["events"]["collisions"]
                | obs["events"]["off_route"]
                | obs["events"]["on_shoulder"]
                | obs["events"]["wrong_way"]
            ):
                pass
            else:
                print("Events: ", obs["events"])
                raise Exception("Episode ended for unknown reason.")

        return obs, wrapped_reward, done, info

    def _reward(self, obs: Dict[str, gym.Space], env_reward: np.float64) -> np.float64:
        reward = 0

        # Penalty for driving off road
        if obs["events"]["off_road"]:
            reward -= 10
            print(f"ENV: Vehicle went off road.")
            return np.float64(reward)

        # Penalty for driving on road shoulder
        if obs["events"]["on_shoulder"]:
            reward -= 10
            print(f"ENV: Vehicle went on road shoulder.")
            return np.float64(reward)

        # Penalty for driving on wrong way
        if obs["events"]["wrong_way"]:
            reward -= 10
            print(f"ENV: Vehicle went wrong way.")
            return np.float64(reward)

        # Penalty for colliding
        if obs["events"]["collisions"]:
            reward -= 10
            print(f"ENV: Vehicle collided.")
            return np.float64(reward)

        # Penalty for driving off route
        if obs["events"]["off_route"]:
            reward -= 10
            print(f"ENV: Vehicle went off route.")
            return np.float64(reward)

        # Reward for distance travelled
        reward += env_reward

        return np.float64(reward)

In [ ]:
The observation space of `intersection-v0` environment is described in `SMARTS/smarts/env/intersection_env.py`. 

In this tutorial, only the top-down rgb image from the observation space is used as input to our reinforcement learning policy. Therefore, a wrapper is built to filter the top-down rgb image observation.  

from typing import Dict

import gym
import numpy as np


class Observation(gym.Wrapper):
    def __init__(self, env: gym.Env):
        super().__init__(env)
        old_space = env.observation_space["rgb"]
        self.observation_space = gym.spaces.Box(
            low=0,
            high=255,
            shape=(old_space.shape[-1],) + old_space.shape[:-1],
            dtype=np.uint8,
        )

    def step(self, action):
        """Steps the environment by one step.

        Args:
            actions (Any): Agent's action.

        Returns:
            Tuple[ np.ndarray, float, bool, Dict[str, Any] ]:
                Observation, reward, done, info, of the agent.
        """
        obs, rewards, dones, infos = self.env.step(action)
        filtered = filter_obs(obs)
        return filtered, rewards, dones, infos

    def reset(self):
        """Resets the environment.

        Returns:
            np.ndarray: Agent's observation after reset.
        """
        obs = self.env.reset()
        filtered = filter_obs(obs)
        return filtered


def filter_obs(obs: Dict[str, gym.Space]) -> np.ndarray:
    rgb = obs["rgb"]

    # Channel first
    rgb = rgb.transpose(2, 0, 1)

    return np.uint8(rgb)

In [ ]:
We proceed to make the `intersection-v0` environment and wrap it with the previously built wrappers. The environment is additionally wrapped with `VecFrameStack`, from Stable Baselines3 library, to give a sense of time to the policy network.

from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack, VecMonitor

def make_env(config: Dict[str, Any]) -> gym.Env:
    # Create environment
    env = gym.make(
        "smarts.env:intersection-v0",
        headless=True,
        visdom=False,
        sumo_headless=True,
        img_meters=config["img_meters"],
        img_pixels=config["img_pixels"],
    )

    # Wrap env with action, reward, and observation wrapper
    env = Info(env=env)
    env = Action(env=env)
    env = Reward(env=env)
    env = Observation(env=env)

    # Check custom environment
    check_env(env)

    # Wrap env with SB3 wrappers
    env = DummyVecEnv([lambda: env])
    env = VecFrameStack(venv=env, n_stack=config["n_stack"], channels_order="first")
    env = VecMonitor(
        venv=env,
        filename=str(config["logdir"]),
        info_keywords=("is_success",),
    )

    return env

In [ ]:
The training and evaluation code is prepared using the Stable Baseline3 API. Training progress is checkpointed using a callback. At the end, the trained agent is saved and evaluated. 